# 处理缺失值

In [1]:
import numpy as np
import pandas as pd

## `None`: `Python`对象类型的缺失值

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

## 这里 dtype=object 表示 NumPy 认为由于这个数组是 Python 对象
## 构成的，因此将其类型判断为 object，对数据的任何操作最终都会在 Python 层面完成
## 但是在进行常见的快速操作时，这种类型比其他原生类型数组要消耗更多的资源：

array([1, None, 3, 4], dtype=object)

In [3]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

# object 类型时，速度较慢；

dtype = object
31.8 ms ± 19.4 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
787 µs ± 845 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)



In [4]:
vals1.sum() # None 值为 object 类型时，常见的操作如 sun(), min() 出现类型错误

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

## `NaN`：数值类型的缺失值

In [6]:
vals2 = np.array([1, np.nan, 3, 4])
vals2.dtype

### np.nan为一特殊的 浮点数

dtype('float64')

In [8]:
# 可以把 NaN 看作是一个数据类病毒——它会将与它接触过的数据同化。
# 无论和 NaN 进行何种操作，最终结果都是NaN
1 + np.nan

nan

In [10]:
0 * np.nan

nan

In [11]:
# 常见操作都是合理的，不会抛出异常，但是并非总是有效
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

In [12]:
# 些特殊的累计函数，它们可以忽略缺失值的影响：
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

## Pandas中`NaN`和`None`的差异

In [13]:
pd.Series([1, np.nan, 2, None])

# 虽然 NaN 与 None 各有各的用处，
# 但是 Pandas 把它们看成是可以等价交换的，在适当的时候会将两者进行替

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

In [14]:
x = pd.Series(range(5), dtype=int)
x

0    0
1    1
2    2
3    3
4    4
dtype: int64

In [15]:
x[0] = None
x

### 注意数据类型的改变！！！！

0    NaN
1    1.0
2    2.0
3    3.0
4    4.0
dtype: float64

>Pandas对缺失值进行强制转换的规则：

|Typeclass     | Conversion When Storing NAs | NA Sentinel Value      |
|--------------|-----------------------------|------------------------|
| ``floating`` | No change                   | ``np.nan``             |
| ``object``   | No change                   | ``None`` or ``np.nan`` |
| ``integer``  | Cast to ``float64``         | ``np.nan``             |
| ``boolean``  | Cast to ``object``          | ``None`` or ``np.nan`` |

> Pandas 中 ***字符串类型的数据通常是用 object 类型存储的。***


## 缺失值的处理

isnull(): 
   - 创建一个布尔类型的掩码标签缺失值。
   
notnull(): 
   - 与 isnull() 操作相反。
   
dropna(): 
   - 返回一个剔除缺失值的数据。       
   
fillna(): 
   - 返回一个填充了缺失值的数据副本。

### 发现缺失值

In [16]:
data = pd.Series([1, np.nan, 'hello', None])
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [17]:
data[data.notnull()]

0        1
2    hello
dtype: object

### 剔除缺失值

In [18]:
data.dropna()

0        1
2    hello
dtype: object

In [19]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [20]:
df.dropna()

# 默认情况下，dropna() 会剔除任何包含缺失值的 整行数据

,0,1,2
1,2.0,3.0,5


In [21]:
df.dropna(axis='columns')

# 可以设置按不同的坐标轴剔除缺失值，
# 比如 axis=1（或axis='columns'）会剔除任何包含缺失值的 整列数据

,2
0,2
1,5
2,6


In [22]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [23]:
df.dropna(axis='columns', how='all')

# 设置 how='all'，这样就只会剔除  全部是缺失值  的行或列 

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [24]:
df.dropna(axis='columns', how='any')

# 默认设置是 how='any'，只要有缺失值就剔除 整行或整列

,2
0,2
1,5
2,6


In [26]:
df.dropna(axis='rows', thresh=3)

# 通过 thresh 参数设置行或列中 **非缺失值**  的最小数量

,0,1,2,3
1,2.0,3.0,5,NaN


## 填充缺失值

In [27]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [28]:
data.fillna(0)

# 指定填充值

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [29]:
data.fillna(method='ffill')

## 用缺失值前面的有效值来从前往后填充（forward-fill）

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [30]:
data.fillna(method='bfill')
## 也可以用缺失值后面的有效值来从后往前填充（back-fill）

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [32]:
df.fillna(method='ffill', axis=1)

# dataframe 填充时需要指定坐标轴参数 axis
# 假如在从前往后填充时，需要填充的缺失值前面没有值，那么它就仍然是缺失值。

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0
